In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c m5-forecasting-accuracy

 98%|█████████████████████████████████████▎| 45.0M/45.8M [00:02<00:00, 20.6MB/s]
100%|██████████████████████████████████████| 45.8M/45.8M [00:02<00:00, 18.4MB/s]


In [5]:
!unzip m5-forecasting-accuracy.zip

Archive:  m5-forecasting-accuracy.zip
  inflating: calendar.csv            
  inflating: sales_train_evaluation.csv  
  inflating: sales_train_validation.csv  
  inflating: sample_submission.csv   
  inflating: sell_prices.csv         


In [1]:
import pandas as pd
import numpy as np
from utils import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
path_m5 = "./"
calendar = pd.read_csv(path_m5+"calendar.csv")
prices = pd.read_csv(path_m5+"sell_prices.csv")
sales_train = pd.read_csv(path_m5+"sales_train_evaluation.csv")
sales_val = pd.read_csv(path_m5+"sales_train_validation.csv")

In [9]:
sales = pd.concat([sales_train, sales_val])

In [10]:
sales_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [11]:
calendar = calendar.rename(columns={"d":"day"})

In [ ]:
calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales = encode_categorical(
    sales, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

prices = encode_categorical(prices, ["item_id", "store_id"]).pipe(
    reduce_mem_usage
)

Mem. usage decreased to  0.07 Mb (66.9% reduction)


In [ ]:
prices_dates = pd.merge(prices, calendar[["wm_yr_wk", "date", "day", "weekday", "wday", "month"]],
                  on="wm_yr_wk", how="left")

In [ ]:
sales_ts = pd.melt(sales,
                 id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                 var_name = 'day', value_name = 'demand')

In [ ]:
del(sales)

In [ ]:
cols_group = ['item_id', 'store_id']
prices_dates['full_price'] = prices_dates.groupby(cols_group)['sell_price'].transform('max')

In [ ]:
prices_dates['discount_pct'] = (prices_dates['full_price']-prices_dates['sell_price'])/prices_dates['full_price']*100

In [ ]:
sales_ts = pd.merge(sales_ts, prices_dates[["day", "item_id", "store_id", "sell_price", "full_price", "discount_pct"]], 
                    on=["day", "item_id", "store_id"])


In [ ]:
df_all = sales_ts.groupby("day").agg({"demand":"sum", 
                                      "sell_price":"mean", 
                                      "full_price":"mean", 
                                      "discount_pct":"mean"}).reset_index()
df_all = pd.merge(df_all, calendar[["day", "date", "weekday", "wday", "month"]], on="day", how="left")
df_all = df_all.rename(columns={"date":"ds", "demand":"y"})
df_all['ds'] = df_all['ds'].astype(np.datetime64)

In [ ]:
df_all = df_all.sort_values("ds")
df_all.to_csv("sales_ms_all.csv", index=False)

In [ ]:
df_cat = sales_ts.groupby(["cat_id", "day"]).agg({"demand":"sum", 
                                      "sell_price":"mean", 
                                      "full_price":"mean", 
                                      "discount_pct":"mean"}).reset_index()
df_cat = pd.merge(df_cat, calendar[["day", "date", "weekday", "wday", "month"]], on="day", how="left")
df_cat = df_cat.rename(columns={"date":"ds", "demand":"y"})
df_cat['ds'] = df_cat['ds'].astype(np.datetime64)

In [ ]:
df_cat = df_cat.sort_values("ds")
df_cat.to_csv("sales_ms_cat.csv", index=False)

In [ ]:
df_cat.head()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df_all["ds"], y=df_all["y"],
                          mode='lines',
                          name='Sales',
                          marker_color='rgb(121,121,121)'))

fig.update_traces(mode='markers+lines')
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout({
    'height': 800,
    'template': 'plotly_white',
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'xaxis_title': 'Fecha',
    'yaxis_title': 'Unidades',
    'yaxis_tickformat': ',.0f',
    'yaxis.rangemode': 'tozero',
    'legend': {'orientation': 'h',
                'yanchor': 'bottom',
                'y': 1.02,
                'xanchor': 'right',
                'x': 1
                }
})